# Exercise: Data Cleaning and Analysis

In practise, data often comes labeled with codes or extreme abbreviations like "Schw_Tr_d_Le_en_W", instead of descriptive column names. Entries are often missing or erroneous, which can introduce errors to machine learning models. Data cleaning serves the purpose of fixing erroneous entries and ensuring the integrity of the dataset, but it does _not_ involve transforming the data in order to prepare it for an algorithm, e.g. via scaling. The exact steps of a data cleaning process depend on the data at hand, but often include making the data humanly interpretable, removing false/incomplete data points, fixing corrupt entries, removing duplicates, etc. 

In [ ]:
# For this exercise, only use pandas
import pandas as pd

##### 1. Load "raw_data.csv" into a dataframe and rename all columns to match _Description_ from Table 1.

In [ ]:
# Load raw data
df_raw = pd.read_csv("raw_data.csv", sep=";")

# Rename all attributes to match the _description_ in Table 1
df = df_raw.rename(columns={
    "od" : "order_date",
    "dd" : "delivery_date",
    "a6" : "salutation",
    "a7" : "date_of_birth",
    "a8" : "state",
    "a9" : "return_shipment"})

##### 2. Correct the data types for all _nominal_ attributes and assign the corresponding labels that are specified under _Comment_ in Table 1.

In [ ]:
# There are multiple ways to to this. We use pd.DataFrame.replace() and pass a dict to it

df["salutation"] = df["salutation"].replace({
    2 : "Company",
    3 : "Mr.",
    4 : "Mrs."}).astype("category")

df["state"] = df["state"].replace({
    1 : "BW",
    2 : "BY",
    3 : "BE",
    4 : "BB",
    5 : "HB",
    6 : "HH",
    7 : "HE",
    8 : "MV",
    9 : "NI",
    10 : "NW",
    11 : "RP",
    12 : "SL",
    13 : "SN",
    14 : "ST",
    15 : "SH",
    16 : "TH"}).astype("category")

df["return_shipment"] = df["return_shipment"].replace({
  0 : "No",
  1 : "Yes"}).astype("category")

##### 3. Correct the data type of the _ordinal_ attribute "size" and assign the corresponding labels specified under _Comment_ in Table 1.

In [ ]:
# Make sure to fix case-sensitivity, otherwise there will be distinct categories created for every different case
df["size"] = df["size"].astype("str").str.upper()  
df["size"] = pd.Series(pd.Categorical(df["size"], categories=["S", "M", "L", "XL", "XXL", "XXXL"], ordered=True))

##### 4. Correct the data types for all _date_ attributes. Split "order_date" into separate columns for "weekday", "year", "month", "day" and "quarter".

In [ ]:
# Use pandas Datetime format for this. Pass errors="coerce" to convert erroneous values into NaT (not a time)
df["order_date"] = pd.to_datetime(df["order_date"])
df["delivery_date"] = pd.to_datetime(df["delivery_date"], errors="coerce", format="%Y-%m-%d")
df["date_of_birth"] = pd.to_datetime(df["date_of_birth"], errors="coerce", format="%Y-%m-%d")

# Make separate columns for order_date
df["order_date_weekday"] = df["order_date"].dt.dayofweek
df["order_date_day"] = df["order_date"].dt.day
df["order_date_month"] = df["order_date"].dt.month
df["order_date_year"] = df["order_date"].dt.year
df["order_date_quarter"] = df["order_date"].dt.quarter

##### 5. Find missing values (NaN, NaT, None), remove or fill these entries (e.g. by mean).

To deal with missing values adequately, it is important to understand what type of data is at hand, and why it is missing. For example, if the date of birth of a customer is not specified, the data point might still contain valuable information about the customer's orders, and it would be a waste to remove the complete data point. In such cases, it can make sense to keep the value as NaN or introduce a default value which makes it apparent that this value was missing.

In [ ]:
# First, locate the columns that have at least one missing value (None, NaN, NaT, and similar).
print(df.isna().any())  # -> delivery_date, price, tax, date_of_birth

In [ ]:
# Fill "price" and "tax" with their column means.
means = {"price" : df["price"].mean(), "tax" : df["tax"].mean()}
df = df.fillna(value=means)

In [ ]:
# Remove rows with missing "delivery_date" entries.
df = df.dropna(subset=["delivery_date"])

### Now that the data is essentially clean, perform some basic analysis on it.

##### 6. Create a new column for "delivery_time" as the difference of "delivery_date" and "order_date". Inspect the created column for errors and label erroneous entries accordingly.

In [ ]:
# Assigning to a new column name will automatically create it
# Pandas supports strucure operations, so simply subtracting columns from each other works
# Access the amount of days via the datetime accessor of the datetime object (.dt.days)
df["delivery_time_days"] = (df["delivery_date"] - df["order_date"]).dt.days

In [ ]:
# pd.DataFrame.where() replaces values where the specified condition is False
df["delivery_time_days"] = df["delivery_time_days"].where(df["delivery_time_days"] > 0)

##### 7. Plot a histogram for the new "delivery_time_days" column. Then discretize its values into the bins "NaN", "<=5d", and ">5d" and store these in a new column "delivery_time_days_discrete". Plot a bar chart for "delivery_time_days_discrete".

In [ ]:
# Plot histogram
df["delivery_time_days"].plot.hist(bins=20)

In [ ]:
# Discretize using pd.cut(), NaN values will be binned into NaN category by default
max_time = df["delivery_time_days"].max()
df["delivery_time_days_discrete"] = pd.cut(df["delivery_time_days"], 
                                           bins=[0, 5, max_time], 
                                           labels=["<=5d", ">5d"])
df["delivery_time_days_discrete"]

In [ ]:
# Plot bar chart
df["delivery_time_days_discrete"].value_counts(sort=False).plot.bar(color=["green", "orange"])

##### 8. Compute the correlation matrix for the numerical attributes. Plot the matrix of the scatterplots. Plot the heatmap of the correlation matrix.

In [ ]:
# Correlation matrix of dataframe
corr = df.corr(numeric_only=True)
print(corr)

In [ ]:
# Plot scatter matrix
matrix_of_scatter_plots = pd.plotting.scatter_matrix(df, figsize=(30,30))

In [ ]:
# Compute correlation matrix of standardized data
df_only_numerical = df.select_dtypes(include="number")
df_normalized = (df_only_numerical - df_only_numerical.mean()) / df_only_numerical.std()
corr_normalized = df_normalized.corr()

In [ ]:
print("Corporate needs you to find the differences between corr matrix and corr_normalized matrix.")

difference = corr - corr_normalized
if (difference < 1e-8).all().all():
    print("--> They're the same matrix.")